In [9]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles
using Random

In [10]:
path = "C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/";

In [11]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity30_.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance30.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"),header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File(string(path,"D_Dsoc.csv"),header=1,delim=",") |> DataFrame
emi_sink = CSV.File(string(path,"emission_sink.csv"),header=1,delim=",") |> DataFrame;

In [12]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi

emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"] .* 0.6          # national CO2 sink (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
# EF_trans = 86/1000000                          #5-LH trucks emit on average 56.6 gCO2/tkm while 5-RD emit 84.0 gCO2/tkm
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
penalty = 5; # ton/yr;

In [13]:
# list = [1,4,5,31,37,38,49];
# list = [1,4,5,11,21,31,37,41,49];
list = [1,3,4,5,9,17,18,31,32,37,38,41,43,44,48,49];
# list = [1];

In [14]:
scen_eff = CSV.File(string(path,"Scenarios/Scen_Efficiency2.csv"), header=1, delim=",") |> DataFrame
scen_eff = Matrix(scen_eff)[:, 2:end]
scen_eff = scen_eff[:, list]
num_omega = size(scen_eff)[2]

scen_dmd = CSV.File(string(path,"Scenarios/Scen_Demand.csv"), header=1, delim=",") |> DataFrame
scen_dmd = Matrix(scen_dmd)[2:end][list]
# scen_dmd = ones(num_omega) .* 452000

Omega = ones(Float64, ncty, num_omega, nproc);  # Omega[ncty,nscena,nproc]
Omega[:, :, 1] = scen_eff;

dmd_fixed = 452000

452000

---

### Solve Subproblem - Get Dual $\pi$ & $\alpha$

In [68]:
function subprob(x_hat, s) # s represents specific scenario     
    
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, unmeet >= 0)

    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5] * scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11] * scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14] * scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15] * scaler[k]) for k in battery for j in 1:ncty] 
    
    cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet == scen_dmd[s])]
    
    
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
        end
        transD[j] = arc_emi  # ncty*1 matrix
    end
    
    
    pro_sink = zeros(AffExpr, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = sum(pro_sink)
    
    
    ############### Obj function ##############
    @objective(model, Min, sum(transD) + unmeet*penalty - proS)
    JuMP.optimize!(model)
    
    qy_hat = JuMP.objective_value(model)
    sub_y = JuMP.value.(y)
    sub_unmeet = JuMP.value.(unmeet)
    
    
    ################## get dual and calculate cuts ################
    dual_pi = zeros(ncty,nproc)

    op_pi = [getdual(con) for con in cstr_op]
    cth_pi = [getdual(con) for con in cstr_cth]
    cell_pi = [getdual(con) for con in cstr_cell]
    noncell_pi = [getdual(con) for con in cstr_noncell]
    battery_pi = [getdual(con) for con in cstr_battery]
    fmk_alp = [getdual(con) for con in cstr_fmk]


    dual_pi[:, cathode] = reshape(cth_pi, ncty, length(cathode)) .* Omega[:,s,cathode] .* repeat(scaler[cathode]', ncty, 1)
    dual_pi[:, cell] = reshape(cell_pi, ncty, length(cell)) .* Omega[:,s,cell] .* repeat(scaler[cell]', ncty, 1)
    dual_pi[:, noncell] = reshape(noncell_pi, ncty, length(noncell)) .* Omega[:,s,noncell] .* repeat(scaler[noncell]', ncty, 1)
    dual_pi[:, battery] = reshape(battery_pi, ncty, length(battery)) .* Omega[:,s,battery] .* repeat(scaler[battery]', ncty, 1)
    dual_pi += reshape(op_pi, ncty, nproc) .* Omega[:,s,:]
    
    dual_alpha = sum(fmk_alp) * scen_dmd[s]

    #######################
    result = Dict(["qyhat"=>qy_hat, "y_opt"=>sub_y, "unmeet_opt"=>sub_unmeet, "pi"=>dual_pi, "alp"=>dual_alpha]);
    return result
    
end

subprob (generic function with 1 method)

In [69]:
function masterprob(cuts)
    model = Model(Gurobi.Optimizer)
    set_silent(model)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, theta >= M)
    
    for k in 1:nproc
        for i in 1:ncty
            @constraint(model, x[i,k] <= 0.8*capacity[!, 2:end][i,k])
        end
    end

    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];
    

    ###########
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    
    
    G = 0
    g = 0
    if cuts != []
        for cut in cuts
            G = cut["gradient"]
            g = cut["intersection"]
            @constraint(model, theta >= sum(x .* G) + g); 
        end
    end
    
    
    @objective(model, Min, sum(proD)+theta)
    JuMP.optimize!(model);
    
    x_hat = JuMP.value.(x)
    theta_hat = JuMP.value.(theta)
    z_lb = JuMP.objective_value(model)
    
    result = Dict(["x_hat"=>x_hat, "z_lb"=>z_lb])
    return result

end

masterprob (generic function with 1 method)

In [70]:
function add_cuts(x)
    G = [] # gradient
    g = [] # intersection
    qy = 0
    
    
    for s in 1:num_omega
        res = subprob(x, s)        
        push!(G, res["pi"])   # gradient
        push!(g, res["alp"])  # intersection
        qy += res["qyhat"]
    end
    
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    
    
    z_hat = sum(proD) + qy/num_omega
    Exp_G = sum(G) / num_omega
    Exp_g = sum(g) / num_omega
    
    cut = Dict(["gradient"=>Exp_G, "intersection"=>Exp_g])
    return cut, z_hat

end

add_cuts (generic function with 1 method)

In [71]:
function main(toler)
    i = 0
    z_ub = 1e8
    x_opt = 0
    
    
    cuts = []
    res0 = masterprob(cuts)
    x_hat = res0["x_hat"]
    z_lb = res0["z_lb"]
    
    
    while (z_ub - z_lb) > toler * min(abs(z_ub), abs(z_lb))
        new_cut, z_hat = add_cuts(x_hat)
        
        if z_hat < z_ub
            x_opt = x_hat
            z_ub = z_hat
        end
        
        push!(cuts, new_cut)
        
        res1 = masterprob(cuts)
        z_lb = res1["z_lb"]
        x_hat = res1["x_hat"]
        
        i += 1
    end
    
    
    result = Dict(["z_lb"=>z_lb, "z_ub"=>z_ub, "x_opt"=>x_opt, "iteration"=>i])
    return result
    
end

main (generic function with 1 method)

----

In [72]:
M = 0
toler = 0.01

0.01

In [73]:
i = 0
z_ub = 1e8
x_opt = 0

cuts = []
res0 = masterprob(cuts)
x_hat = res0["x_hat"]
z_lb = res0["z_lb"]

Academic license - for non-commercial use only - expires 2024-12-26


0.0

In [74]:
new_cut, z_hat = add_cuts(x_hat)
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)

res1 = masterprob(cuts)
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

30×15 Matrix{Float64}:
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0  22032.7  22581.4       0.0    0.0        0.0
     0.0   2480.0      0.0      0.0       0.0    0.0        0.0
 16165.7      0.0      0.0      0.0       0.0    1.47016e5  0.0
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0  20143.6      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0  …  588.062  0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     ⋮                               ⋱                      
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0   

In [75]:
(z_ub - z_lb) / min(abs(z_ub), abs(z_lb))

0.5324093348716853

In [76]:
new_cut, z_hat = add_cuts(x_hat)
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)

res1 = masterprob(cuts)
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

30×15 Matrix{Float64}:
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0  22032.7  22581.4       0.0    0.0        0.0
     0.0   2480.0      0.0      0.0       0.0    0.0        0.0
 16165.7      0.0      0.0      0.0       0.0    1.47016e5  0.0
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0  20143.6      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0  …  588.062  0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     ⋮                               ⋱                      
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0   

In [77]:
(z_ub - z_lb) / min(abs(z_ub), abs(z_lb))

0.10892959372836099

In [78]:
new_cut, z_hat = add_cuts(x_hat)
if z_hat < z_ub
    x_opt = x_hat
    z_ub = z_hat
end

push!(cuts, new_cut)

res1 = masterprob(cuts)
z_lb = res1["z_lb"]
x_hat = res1["x_hat"]

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

30×15 Matrix{Float64}:
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0  22032.7  22581.4       0.0    0.0        0.0
     0.0   2480.0      0.0      0.0       0.0    0.0        0.0
 16165.7      0.0      0.0      0.0       0.0    1.47016e5  0.0
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0  20143.6      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0  …  588.062  0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     ⋮                               ⋱                      
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0   

In [79]:
(z_ub - z_lb) / min(abs(z_ub), abs(z_lb))

0.10892959372836099

In [82]:
x_opt

30×15 Matrix{Float64}:
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0  22032.7  22581.4       0.0    0.0        0.0
     0.0   2480.0      0.0      0.0       0.0    0.0        0.0
 16165.7      0.0      0.0      0.0       0.0    1.47016e5  0.0
     0.0      0.0      0.0      0.0  …    0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0  20143.6      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0  …  588.062  0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     ⋮                               ⋱                      
     0.0      0.0      0.0      0.0       0.0    0.0        0.0
     0.0      0.0   

In [85]:
df_xopt = DataFrame(x_opt,:auto);

In [86]:
CSV.write(string(path,"/result/xopt.csv"), df_xopt)

"C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/src/deterministic/data/xopt.csv"

---

In [27]:
function gen_fea(x_hat)
    ls = []
    for s in 1: num_omega
        model = Model(Gurobi.Optimizer)
        set_silent(model)
        @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
        @variable(model, unmeet >= 0)

        cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= Omega[i,s,k] * x_hat[i,k]) for k in 1:nproc for i in 1:ncty]
        cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
        cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,11]*scaler[k]) for k in cell for j in 1:ncty]
        cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
        cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= Omega[j,s,k] * x_hat[j,15]*scaler[k]) for k in battery for j in 1:ncty] 

        cstr_fmk = [@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) + unmeet >= scen_dmd[s])]
        
        transD = Vector{AffExpr}(undef, ncty)
        for j in 1:ncty
            arc_emi = 0
            for i in 1:ncty
                amount = sum(y[i,j,k] for k in 1:nproc)
                arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            end
            transD[j] = arc_emi  # ncty*1 matrix
        end


        pro_sink = zeros(AffExpr, ncty, nproc)
        for k in 1:nproc
            for i in 1:ncty
                pro_sink[i,k] = (sum(y[i,j,k] for j in 1:ncty)) * regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
            end
        end
        proS = sum(pro_sink)

        @objective(model, Min, sum(transD) + unmeet*penalty - proS)
        JuMP.optimize!(model)

        if termination_status(model) == MOI.OPTIMAL
            push!(ls, s)
        end
    end
    
    return ls
    
end

gen_fea (generic function with 1 method)

In [43]:
ls = gen_fea(x_hat)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

16-element Vector{Any}:
  1
  3
  4
  5
  9
 17
 18
 31
 32
 37
 38
 41
 43
 44
 48
 49

In [36]:
xhat1 = zeros(30,15)
gen_fea(xhat1)

Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only - expires 2024-12-26
Academic license - for non-commercial use only -

50-element Vector{Any}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50